# TANGO1 project

## achievment so far

In [ ]:
./../pipelines/aln_generation_colouring.nf --INPUT frequency_trainig_and_positive_set4.fastaoneline \\
    --TRIMM 70,30 --PLP_DIR ../all_phobius_results/good_pp_vert_originalModel/plps/  \\
    --PHOB_STDOUT ../all_phobius_results/good_pp_vert_originalModel/phobius_output_om_frequency_trainig_and_positive_set4.txt \\
    -resume -profile bovreg -process.cpus 3

just an example to see ifit is possibleto share